In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from copy import deepcopy

class Normalizer:
    """
    The Great Normalizer
    """
    mins: np.ndarray
    maxs: np.ndarray
    span: np.ndarray
    mids: np.ndarray

    def __init__(self, reference_inputs: np.ndarray):
        self.mins = np.min(reference_inputs, 0)
        self.maxs = np.max(reference_inputs, 0)
        self.span = self.maxs - self.mins
        # mids = span / 2 + mins
        # and thus:
        # mids = (maxs + mins) / 2
        self.mids = (self.maxs + self.mins) / 2

    def scale(self, set: np.ndarray):
        # k0 = (reference_inputs[0] - mids) / (span / 2)
        # and thus:
        # k0 = 2*(reference_inputs[0] - mids) / span
        return 2 * np.subtract(set, self.mids) / self.span


def sig(x):
    return 1 / (1 + np.exp(-x))

def shp(x):
    return x.reshape(len(x), -1)
def ytransform(y):
    arr = np.zeros((len(y), 10))
    rng = np.array([np.arange(0, len(y)), y]).T
    for i, v in rng:
        arr[i,v] = 1
    return arr


def polynom(inputs: np.ndarray, degree=2):
    if degree < 2:
        raise Exception("Degree should be at least 2")

    transposed = inputs.T
    base = len(transposed)

    pols = []
    newgen = []
    for i in range(base):
        newgen.append([i])
    pols.extend(newgen)

    for deg in range(1, degree):
        lastgen = newgen
        newgen = []
        for p in lastgen:
            for i in range(p[-1], base):
                pnext = [*p, i]
                pnext.sort()
                newgen.append(pnext)
        pols.extend(newgen)

    new_inputs = []
    for p in pols:
        new_inputs.append(deepcopy(transposed[p[0]]))
        for i in p[1:]:
            new_inputs[-1] *= transposed[i]
    return np.array(new_inputs).T

dataset = np.array([
    [2104, 5, 1, 45, 460],
    [1416, 3, 2, 40, 232],
    [1534, 3, 2, 30, 315],
    [1600, 4, 2, 30, 389],
    [852, 2, 1, 36, 178],
])
x_train = polynom(dataset[:, :-1])

normer = Normalizer(x_train)
x_train1 = normer.scale(x_train)

norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(x_train)
x_train2 = np.array(norm_l(x_train))

y_train = dataset[:, -1:]

np.set_printoptions(precision=1, suppress=True)
print("CUSTOM NORMALIZER:")
print(x_train1)
print("TF NORMALIZER:")
print(x_train2)

CUSTOM NORMALIZER:
[[ 1.   1.  -1.   1.   1.   1.   0.1  1.   1.   0.   1.  -1.  -0.6  1. ]
 [-0.1 -0.3  1.   0.3 -0.3 -0.4  0.7 -0.2 -0.5  0.3 -0.4  1.   1.   0.2]
 [ 0.1 -0.3  1.  -1.  -0.1 -0.3  0.9 -0.5 -0.5  0.3 -0.8  1.   0.1 -1. ]
 [ 0.2  0.3  1.  -1.  -0.   0.1  1.  -0.5  0.1  1.  -0.4  1.   0.1 -1. ]
 [-1.  -1.  -1.  -0.2 -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -0.3]]
TF NORMALIZER:
[[ 1.5  1.6 -1.2  1.5  1.7  1.7 -0.4  1.8  1.7 -0.2  1.9 -1.2 -0.7  1.6]
 [-0.2 -0.4  0.8  0.7 -0.3 -0.4  0.5  0.1 -0.5  0.3 -0.1  0.8  1.6  0.6]
 [ 0.1 -0.4  0.8 -1.1 -0.1 -0.3  0.8 -0.4 -0.5  0.3 -0.7  0.8  0.3 -1. ]
 [ 0.2  0.6  0.8 -1.1  0.1  0.3  0.9 -0.3  0.5  1.3 -0.1  0.8  0.3 -1. ]
 [-1.6 -1.4 -1.2 -0.  -1.4 -1.3 -1.8 -1.1 -1.2 -1.7 -1.  -1.2 -1.3 -0.1]]
